# October 17th Meeting

## Implementation of 4th Order Operators

In [3]:
function Dx_p4(y_in, Nx, Ny, h)
	d  = [1/12 -2/3 0 2/3 -1/12]
    bd = [-24/17  59/34  -4/17  -3/34  0     0;
           -1/2    0      1/2    0     0     0;
            4/43 -59/86   0     59/86 -4/43  0;
            3/98   0    -59/98   0    32/49 -4/49]
	N = Nx*Ny
	y_out = zeros(N)

	for row_num = 1:size(bd)[1]
		idx = (row_num - 1)*Ny+1:(row_num)*Ny
		y_out[idx] = (bd[row_num,1] * y_in[idx .+ (1 - row_num)*Ny] + bd[row_num,2] * y_in[idx .+ Ny*(2-row_num)] + bd[row_num,3] * y_in[idx .+ Ny*(3-row_num)] + bd[row_num,4] * y_in[idx .+ Ny*(4-row_num)] + bd[row_num,5] * y_in[idx .+ Ny*(5-row_num)] + bd[row_num,6] * y_in[idx .+ Ny*(6-row_num)]) ./ h
	end

	idx1 = (size(bd)[1])*Ny+1:N-(size(bd)[1])*Ny
	y_out[idx1] = (d[1]*y_in[idx1 .- 2*Ny] + d[2]*y_in[idx1 .- Ny] + d[3]*y_in[idx1] + d[4]*y_in[idx1 .+ Ny] + d[5] * y_in[idx1 .+ 2*Ny]) ./ (h)

	for row_num = 1:size(bd)[1]
		idx2 = N-(row_num)*Ny+1:N .- (row_num - 1)*Ny
		y_out[idx2] = -(bd[row_num,1] * y_in[idx2 .+ (row_num - 1)*Ny] + bd[row_num,2] * y_in[idx2 .- Ny*(2-row_num)] + bd[row_num,3] * y_in[idx2 .- Ny*(3-row_num)] + bd[row_num,4]*y_in[idx2 .- Ny*(4-row_num)] + bd[row_num,5] * y_in[idx2 .- Ny*(5-row_num)] + bd[row_num,6] * y_in[idx2 .- Ny*(6-row_num)]) ./ h
	end

	return y_out
end

Dx_p4 (generic function with 1 method)

```jl
julia> y_out_1 = kron(D,I_x) * y_in;
julia> y_out_2 = Dx_p4(y_in,Nx,Ny,h);


julia> y_out_1 ≈ y_out_2
true

julia> reshape(y_out_1 - y_out_2,11,11)
11×11 Array{Float64,2}:
  0.0           4.44089e-16   2.22045e-16   8.88178e-16   0.0          …  -8.88178e-16  -8.88178e-16   0.0          -3.33067e-16
 -4.44089e-16  -4.44089e-16   2.22045e-16  -1.77636e-15   8.88178e-16      3.33067e-16   0.0           8.88178e-16   0.0
 -3.55271e-15  -8.88178e-16   8.88178e-16   0.0           2.22045e-16      1.77636e-15  -1.77636e-15   0.0           0.0
  0.0           0.0           1.77636e-15  -1.77636e-15  -1.77636e-15      1.77636e-15  -1.77636e-15  -1.77636e-15   3.55271e-15
  0.0           0.0          -1.77636e-15   0.0          -3.19189e-16     -4.44089e-16   0.0           0.0          -3.55271e-15
  3.55271e-15   0.0           0.0           0.0           0.0          …   0.0           1.77636e-15   8.88178e-16   4.44089e-16
  0.0           1.77636e-15   1.11022e-16   4.44089e-16  -4.44089e-16      8.88178e-16   3.55271e-15   0.0           0.0
  8.88178e-16  -1.77636e-15   0.0          -1.08247e-15   0.0              0.0           0.0           8.88178e-16  -3.55271e-15
 -3.55271e-15   0.0           4.44089e-16  -4.44089e-16  -8.88178e-16      0.0           2.66454e-15  -1.77636e-15   0.0
  7.10543e-15   0.0           7.10543e-15  -1.77636e-15  -7.77156e-16      2.77556e-16   8.88178e-16   8.88178e-16   0.0
  0.0           0.0           0.0           8.88178e-16   0.0          …   1.77636e-15   0.0           0.0           1.77636e-15
```

## Converting to GPU kernels
### More efficient way to store `d` and `bd` as `p` increases
Probably constant memory
```jl
 elseif p == 10
    bhinv = [18289152000/5261271563 1828915200/2881040311 406425600/52175551 6096384/11662993 87091200/50124587 72576000/50124587 87091200/148333439 152409600/63867949 16257024/20608675 1828915200/1704508063 18289152000/18425967263];

    d = [-1/1260,5/504,-5/84,5/21,-5/6,0,5/6,-5/21,5/84,-5/504,1/1260];

    bd = [  -1.7380923775745425e+00   2.3557601935237220e+00  -1.5328406598563976e-01  -5.7266565770416333e-01  -1.8308103515008173e-01   1.8186748267946842e-01   2.0034232582598244e-01   2.2678007363666621e-02  -1.1782459320459637e-01  -3.0591175636402144e-02   3.4890895862586133e-02   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00;
            -4.3020203737210871e-01   0.0000000000000000e+00   1.1837297346927406e-01   3.3928601158526644e-01   1.3241927733034406e-01  -8.7495003780608913e-02  -1.1750484124279399e-01  -1.6401912273575153e-02   6.2537843443041474e-02   1.7143274696828435e-02  -1.8155585855667674e-02   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00;
             3.4348531361887280e-01  -1.4525207124434036e+00   0.0000000000000000e+00   2.9011513992277767e+00  -2.2419288742360557e+00  -5.4662873578741478e-01   1.2908050607446131e+00   6.1514504292452719e-02  -4.2442625460011202e-01   1.5579158905288801e-02   5.2969140277981920e-02   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00;
             8.6111387816878188e-02  -2.7937273515056432e-01  -1.9467880944770807e-01   0.0000000000000000e+00   2.0170150914578375e-01   2.4269917331475005e-01  -7.7261988327590472e-02   5.0649247607525059e-02  -7.4775049946661561e-03  -4.0978487203372188e-02   1.8608207238964152e-02   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00;
             9.1509035082611684e-02  -3.6243526359648576e-01   5.0007055839856984e-01  -6.7045605191055857e-01   0.0000000000000000e+00  -1.7807807859119628e-02   7.5000761407401195e-01  -2.2979723229714316e-01  -1.2521154324370892e-01   6.8278284106004450e-02  -4.1575927541817690e-03   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00;
            -7.5752056274147259e-02   1.9956355926115746e-01   1.0160630736447970e-01  -6.7227694623145351e-01   1.4839839882599690e-02   0.0000000000000000e+00   5.4091068834671807e-01  -1.2712520372174399e-01  -8.9292453564020990e-02   1.6181541970619609e-01  -5.4289154769785249e-02   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00;
            -3.3838029883391296e-02   1.0867927550524317e-01  -9.7293058702223670e-02   8.6783825404790446e-02  -2.5344131542932297e-01  -2.1934035945002228e-01   0.0000000000000000e+00   2.7184438867288430e-01   1.9102691945078512e-01  -4.8646826827046824e-02  -6.2407959378425991e-03   4.6597719614658163e-04   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00;
            -1.5567948806367624e-02   6.1656604470023607e-02  -1.8844858059892756e-02  -2.3122780265804038e-01   3.1560994521078772e-01   2.0951677187991255e-01  -1.1048784865195491e+00   0.0000000000000000e+00   1.1823059621092409e+00  -5.3610400867086083e-01   1.5931375952374752e-01  -2.3673846172827626e-02   1.8939076938262100e-03   0.0000000000000000e+00   0.0000000000000000e+00   0.0000000000000000e+00;
             2.6737701764454301e-02  -7.7712278574126673e-02   4.2981266272823705e-02   1.1284579710276557e-02   5.6847566375570611e-02   4.8647834370398067e-02  -2.5665536068472994e-01  -3.9083324869946684e-01   0.0000000000000000e+00   6.5716944195909766e-01  -1.5822272208022428e-01   4.6954983762905661e-02  -7.8258306271509429e-03   6.2606645017207550e-04   0.0000000000000000e+00   0.0000000000000000e+00;
             9.4425181052687698e-03  -2.8976375375532045e-02  -2.1459742428921558e-03   8.4117843695442701e-02  -4.2165149106440383e-02  -1.1991463562335723e-01   8.8902467992349743e-02   2.4105392677971343e-01  -8.9388344421253152e-01   0.0000000000000000e+00   8.6496680152924643e-01  -2.5547312415382800e-01   6.3868281038457000e-02  -1.0644713506409501e-02   8.5157708051276015e-04   0.0000000000000000e+00;
            -9.9625965676187218e-03   2.8387641187789508e-02  -6.7495090936003027e-03  -3.5335033597892078e-02   2.3750992019053968e-03   3.7216380474824604e-02   1.0550378667904333e-02  -6.6265458456725809e-02   1.9908619649258188e-01  -8.0014409359906680e-01   0.0000000000000000e+00   8.2714572225493910e-01  -2.3632734921569687e-01   5.9081837303924217e-02  -9.8469728839873684e-03   7.8775783071898962e-04];
```


![Constant Memory](graphics/constant_memory.png)


### Better way to rewrite matrix-free version
1. I use one for-loop in this implementations, I manually hardcode 1 to 6 which is the number of columns of `bd`.
2. For higher-order matrix-free operators, we might want to use two for loops to implement kernels. 
3. How to rewrite these for loops in "threads" language might be challenging.

## Research Papers

### GPU-acceleration of A High Order Finite Difference Code Using Curvilinear Coordinates
This paper doesn't use matrix-free methods. The key algorithm is Sparse matrix Vector Multiplication. Which makes "High Order" not very different frm "Low Order".

### [High-Order Matrix-Free Imcompressible Flow Solvers with GPU Acceleration and Low-order Refined Preconditioners](https://arxiv.org/pdf/1910.03032.pdf)
This paper uses High-order matrix-free methods, which might offers direct help to my work.
